We are giving a score for each movie and to each user a unique one.

In [13]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
#import lightgbm as lgb

from lightgbm.sklearn import LGBMRanker
import matplotlib.pyplot as plt

In [3]:
df1=pd.read_csv('movies.csv')
df2=pd.read_csv('ratings.csv')

In [5]:
print(df2.shape)
print(df1.shape)

(25000095, 4)
(62423, 3)


In [6]:
df2.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [10]:
print(df2['userId'].nunique())
print(df2['movieId'].nunique())
print(162541 * 59047)


162541
59047
9597558427


In [14]:
df2.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [32]:
df22=df2.drop(['timestamp'],axis=1)
df22

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5
...,...,...,...
25000090,162541,50872,4.5
25000091,162541,55768,2.5
25000092,162541,56176,2.0
25000093,162541,58559,4.0


In [41]:
n=len(df2)
lis=list(range(0,n))
random.seed(2024)
random.shuffle(lis)
new=df2.iloc[lis[0:3000]]
new

,userId,movieId,rating,timestamp
2197221,14635,590,3.0,843583397
24990561,162508,47634,2.0,1365194093
5619793,36413,440,3.0,839512088
9201713,59973,69306,2.5,1272971505
7812860,50769,141,3.0,1449797467
...,...,...,...,...
11670224,75655,367,4.0,834345062
3259628,21528,590,4.0,1080222243
7103336,46055,2770,4.0,941787655
22889453,148738,6754,4.0,1081024320


In [42]:
new=new.merge(df1,on='movieId',how='left')
new

,userId,movieId,rating,timestamp,title,genres
0,14635,590,3.0,843583397,Dances with Wolves (1990),Adventure|Drama|Western
1,162508,47634,2.0,1365194093,36 Quai des Orfèvres (Department 36) (2004),Action|Crime|Drama|Thriller
2,36413,440,3.0,839512088,Dave (1993),Comedy|Romance
3,59973,69306,2.5,1272971505,"Taking of Pelham 1 2 3, The (2009)",Crime|Drama|Thriller
4,50769,141,3.0,1449797467,"Birdcage, The (1996)",Comedy
...,...,...,...,...,...,...
2995,75655,367,4.0,834345062,"Mask, The (1994)",Action|Comedy|Crime|Fantasy
2996,21528,590,4.0,1080222243,Dances with Wolves (1990),Adventure|Drama|Western
2997,46055,2770,4.0,941787655,Bowfinger (1999),Comedy
2998,148738,6754,4.0,1081024320,Underworld (2003),Action|Fantasy|Horror


In [43]:
from sklearn.preprocessing import MultiLabelBinarizer

In [48]:
enc=MultiLabelBinarizer()
gen=new['genres'].str.split('|').to_list()
gen[0:5]



[['Adventure', 'Drama', 'Western'],
 ['Action', 'Crime', 'Drama', 'Thriller'],
 ['Comedy', 'Romance'],
 ['Crime', 'Drama', 'Thriller'],
 ['Comedy']]

In [56]:
tran=enc.fit_transform(gen)
len(tran)
enc.classes_
len(tran[0])


20

Now I will use these twenty columns and ratings by each user to recommend the movies by giving a score.

In [ ]:
cat=pd.DataFrame(data=tran,columns=enc.classes_)
cat
fre=pd.concat([new,cat],axis=1)
fre


,userId,movieId,rating,timestamp,title,genres,(no genres listed),Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,14635,590,3.0,843583397,Dances with Wolves (1990),Adventure|Drama|Western,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,162508,47634,2.0,1365194093,36 Quai des Orfèvres (Department 36) (2004),Action|Crime|Drama|Thriller,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,36413,440,3.0,839512088,Dave (1993),Comedy|Romance,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,59973,69306,2.5,1272971505,"Taking of Pelham 1 2 3, The (2009)",Crime|Drama|Thriller,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,50769,141,3.0,1449797467,"Birdcage, The (1996)",Comedy,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,75655,367,4.0,834345062,"Mask, The (1994)",Action|Comedy|Crime|Fantasy,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2996,21528,590,4.0,1080222243,Dances with Wolves (1990),Adventure|Drama|Western,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2997,46055,2770,4.0,941787655,Bowfinger (1999),Comedy,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2998,148738,6754,4.0,1081024320,Underworld (2003),Action|Fantasy|Horror,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


Predicting score from ratings alone using svd

In [75]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [76]:
fcol=['userId', 'movieId', 'rating']

In [77]:
read=Reader(rating_scale=(0,5))
data=Dataset.load_from_df(fre[fcol],read)

In [84]:
trainset, testset = train_test_split(data, test_size=0.2)

# Initialize SVD model
model = SVD()

# Train model on training data
model.fit(trainset)

# Evaluate on test data
predictions = model.test(testset)


In [94]:
pred=model.predict(36413,590)
print(pred.est)

3.568857463818685


In [95]:
from surprise import accuracy
rmse=accuracy.rmse(predictions)
amse=accuracy.mae(predictions)

RMSE: 1.0686
MAE:  0.8525


In [89]:
predictions[0]

Prediction(uid=57608, iid=7318, r_ui=2.0, est=3.602554498091307, details={'was_impossible': False})

Now predict scores based on genres(cosine similarity). For that we have to first create  content preference vetors

In [98]:
for i in enc.classes_:
    fre[i]=fre[i]*fre['rating']
fre

,userId,movieId,rating,timestamp,title,(no genres listed),Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,14635,590,3.0,843583397,Dances with Wolves (1990),0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,162508,47634,2.0,1365194093,36 Quai des Orfèvres (Department 36) (2004),0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,36413,440,3.0,839512088,Dave (1993),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,59973,69306,2.5,1272971505,"Taking of Pelham 1 2 3, The (2009)",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0
4,50769,141,3.0,1449797467,"Birdcage, The (1996)",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,75655,367,4.0,834345062,"Mask, The (1994)",0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2996,21528,590,4.0,1080222243,Dances with Wolves (1990),0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2997,46055,2770,4.0,941787655,Bowfinger (1999),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2998,148738,6754,4.0,1081024320,Underworld (2003),0.0,4.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now finding the aggregate of each genre liked per user

In [99]:
contscore=fre.groupby('userId')[list(enc.classes_)].mean()

In [113]:
contscore.values

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 2.5, ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 5. , 0. , 0. ],
       [0. , 3. , 3. , ..., 3. , 0. , 0. ],
       [0. , 2. , 0. , ..., 2. , 0. , 0. ]])

In [114]:
fre[enc.classes_]

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2996,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2997,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2998,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
from sklearn.metrics.pairwise import cosine_similarity
cs=cosine_similarity(contscore.values,fre[enc.classes_].values)

In [105]:
cs

array([[0.        , 0.        , 0.        , ..., 0.        , 0.57735027,
        0.        ],
       [0.33333333, 0.        , 0.        , ..., 0.        , 0.33333333,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.40824829,
        0.        ],
       ...,
       [0.28867513, 0.75      , 0.35355339, ..., 0.5       , 0.        ,
        0.5       ],
       [0.33333333, 0.57735027, 0.        , ..., 0.        , 0.33333333,
        0.        ],
       [0.28867513, 1.        , 0.        , ..., 0.        , 0.28867513,
        0.5       ]])

In [108]:
cont_score=pd.DataFrame(cs,index=contscore.index,columns=fre['movieId'])

In [110]:
cont_score

movieId,590,47634,440,69306,141,653,1394,224,2012,272,...,36,356,3264,1104,1296,367,590,2770,6754,1954
userId,,,,,,,,,,,,,,,,,,,,,
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.577350,0.0,0.000000,0.000000,0.000000,0.000000,0.577350,0.0
43,0.333333,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.288675,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.288675,0.333333,0.000000,0.333333,0.0
87,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.353553,0.000000,...,0.000000,0.000000,0.408248,0.0,0.000000,0.000000,0.000000,0.000000,0.408248,0.0
101,0.000000,0.000000,1.000000,0.000000,0.707107,0.000000,0.707107,0.816497,0.353553,0.500000,...,0.000000,0.707107,0.408248,0.0,0.500000,0.353553,0.000000,0.707107,0.000000,0.0
139,0.577350,0.500000,0.000000,0.577350,0.000000,0.000000,0.000000,0.577350,0.000000,0.707107,...,0.707107,0.500000,0.000000,1.0,0.707107,0.000000,0.577350,0.000000,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162326,0.000000,0.000000,1.000000,0.000000,0.707107,0.000000,0.707107,0.816497,0.353553,0.500000,...,0.000000,0.707107,0.408248,0.0,0.500000,0.353553,0.000000,0.707107,0.000000,0.0
162330,0.000000,0.353553,0.500000,0.408248,0.707107,0.000000,0.707107,0.408248,0.353553,0.500000,...,0.500000,0.353553,0.408248,0.0,0.000000,0.707107,0.000000,0.707107,0.000000,0.0
162457,0.288675,0.750000,0.353553,0.866025,0.500000,0.000000,0.500000,0.577350,0.250000,0.707107,...,0.707107,0.500000,0.288675,0.5,0.353553,0.500000,0.288675,0.500000,0.000000,0.5


In [116]:
from itertools import product
mov=fre['movieId'].unique()
user=fre['userId'].unique()
# colscore=pd.DataFrame(index=user,columns=mov)
# for i in user:
#     for j in mov:
#         ped=model.predict(i,j)
#         colscore.loc[i,j]=pred


predictions = [
    (uid, mid, model.predict(uid, mid).est)
    for uid, mid in product(user,mov)
]

# Convert to DataFrame
pred_df = pd.DataFrame(predictions, columns=['userId', 'movieId', 'rating'])


In [120]:
user_item_matrix = pred_df.pivot(index="userId", columns="movieId", values="rating")

In [ ]:
user_item_matrix/=5


movieId,1,2,3,5,6,7,10,12,14,16,...,187595,189893,190939,190949,192379,192419,192561,195671,196997,204698
userId,,,,,,,,,,,,,,,,,,,,,
9,0.690601,0.716260,0.699288,0.699288,0.725980,0.698070,0.699288,0.705361,0.699288,0.670434,...,0.700792,0.668102,0.644768,0.661492,0.710432,0.748331,0.699192,0.725633,0.652169,0.730699
43,0.699835,0.698848,0.694344,0.694344,0.708633,0.692172,0.694344,0.701271,0.694344,0.738311,...,0.714898,0.662011,0.690536,0.664415,0.675374,0.717436,0.686946,0.710137,0.629573,0.756511
87,0.753647,0.769049,0.733477,0.733477,0.727848,0.733831,0.733477,0.758712,0.733477,0.768178,...,0.704894,0.701960,0.746402,0.725755,0.768517,0.737831,0.753679,0.734364,0.671386,0.784806
101,0.697095,0.741200,0.713025,0.713025,0.708633,0.702711,0.713025,0.750764,0.713025,0.681288,...,0.697241,0.713591,0.724194,0.717345,0.746420,0.740707,0.707912,0.713532,0.626250,0.775601
139,0.730116,0.680461,0.701629,0.701629,0.708830,0.675640,0.701629,0.678219,0.701629,0.720847,...,0.713405,0.679314,0.717240,0.657661,0.703634,0.739883,0.696671,0.699912,0.642667,0.775611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162326,0.698032,0.680924,0.698561,0.698561,0.671374,0.671338,0.698561,0.754943,0.698561,0.690191,...,0.728720,0.658320,0.680196,0.749768,0.645156,0.720333,0.657677,0.719554,0.641183,0.734230
162330,0.715457,0.705632,0.700959,0.700959,0.717818,0.682949,0.700959,0.699040,0.700959,0.701786,...,0.723628,0.692479,0.675655,0.668836,0.721038,0.760511,0.715542,0.701843,0.634769,0.748272
162457,0.724654,0.763521,0.729821,0.729821,0.745237,0.696930,0.729821,0.727073,0.729821,0.743180,...,0.718887,0.683234,0.714352,0.703902,0.754271,0.709450,0.764723,0.761200,0.667192,0.776147


In [126]:
cont_score=cont_score[sorted(cont_score.columns)]

In [130]:
fin=cont_score.loc[:,~cont_score.columns.duplicated()]

In [131]:
fin

movieId,1,2,3,5,6,7,10,12,14,16,...,187595,189893,190939,190949,192379,192419,192561,195671,196997,204698
userId,,,,,,,,,,,,,,,,,,,,,
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.0,0.000000,...,0.000000,0.707107,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000
43,0.774597,1.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,...,0.577350,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000
87,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.000000,...,0.353553,0.500000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.353553,0.000000
101,0.316228,0.000000,1.000000,0.707107,0.000000,1.000000,0.000000,0.500000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000
139,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.707107,...,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.00000,1.0,0.500000,0.577350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162326,0.316228,0.000000,1.000000,0.707107,0.000000,1.000000,0.000000,0.500000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000
162330,0.316228,0.000000,0.500000,0.707107,0.408248,0.500000,0.000000,0.500000,0.0,0.500000,...,0.000000,0.000000,0.408248,0.0,0.0,0.0,0.00000,0.0,0.000000,0.408248
162457,0.223607,0.000000,0.353553,0.500000,0.577350,0.353553,0.288675,0.353553,0.5,0.707107,...,0.000000,0.353553,0.577350,0.0,0.5,0.0,0.50000,0.5,0.500000,0.866025


This is the final matrix where i am combining the 2 calculated scores in a proportion of 0.6(SVD),0.4(genre) 

In [132]:
net=user_item_matrix*0.4 + fin * 0.6 

In [136]:
map=dict(zip(df1['movieId'].values,df1['title'].values))

In [141]:
def rec (user):
    rowint=net.loc[user]
    k=rowint.nlargest(5).index.tolist()
    for i in k :
        print(f'{map[i]}:{rowint[i]}')

In [142]:
rec(9)

Halloween (1978):0.9012563166849894
Hellraiser III: Hell on Earth (1992):0.8952015249455318
The Witch (2015):0.8875671013166585
The Autopsy of Jane Doe (2016):0.8858383007068421
Horror of Dracula (Dracula) (1958):0.8827838746022412
